In [1]:
#Célula 1: Instalação de bibliotecas adicionais
%pip install category_encoders xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Célula 2: Importações
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from shapash import SmartExplainer

#Configuração de visualização
pd.set_option('display.max_columns', None)

c:\XAILabProcess\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Célula 3: Carregamento e Preparação Rápida - Notebook independente

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
df = pd.read_csv(url)

#Renomeando para Português (Facilitando a comunicação com o operador)
mapa_nomes = {
    'Type': 'Tipo_Equipamento',
    'Air temperature [K]': 'Temp_Ar',
    'Process temperature [K]': 'Temp_Processo',
    'Rotational speed [rpm]': 'Rotacao_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Desgaste_Ferramenta_min',
    'Machine failure': 'Falha'
}
df = df.rename(columns=mapa_nomes)

#Removendo colunas desnecessárias e data leakage
df_model = df.drop(['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

#Separando X (Features) e y (Target)
X = df_model.drop('Falha', axis=1)
y = df_model['Falha']

In [4]:
#Célula 4: Divisão Treino e Teste - Validação do modelo

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [5]:
#Célula 5: Codificação de Variáveis Categóricas

encoder = OrdinalEncoder(cols=['Tipo_Equipamento'])
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

In [6]:
#Célula 6: Treinamento do Modelo XGBoost - "Caixa-Preta" (Black Box)

model = XGBClassifier(
    n_estimators=100,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train_enc, y_train)

print("Modelo treinado com sucesso!")

Modelo treinado com sucesso!


In [7]:
#Célula 7: Configuração do Shapash (A Camada de XAI)

#Traduzindo a matemática do modelo para a visão do operador

#Dicionário para o dashboard
features_dict = {
    'Temp_Ar': 'Temperatura do Ambiente',
    'Temp_Processo': 'Temperatura do Processo',
    'Rotacao_rpm': 'Velocidade de Rotação',
    'Torque_Nm': 'Torque do Motor',
    'Desgaste_Ferramenta_min': 'Tempo de Uso da Ferramenta',
    'Tipo_Equipamento': 'Categoria da Máquina'
}

#Instanciando o SmartExplainer
xpl = SmartExplainer(
    model=model,
    preprocessing=encoder, #Passando o encoder para ele reverter os números para texto (L, M, H) no gráfico
    features_dict=features_dict
)

In [8]:
#Célula 8: Compilação das Explicações - Cálculo das contribuições (SHAP/LIME)

xpl.compile(
    x=X_test_enc,
    y_target=y_test #Comparação: Realidade x Previsão
)

INFO: Shap explainer type - <shap.explainers._exact.ExactExplainer object at 0x000001AE4B4D48B0>


ExactExplainer explainer: 3001it [00:49, 60.71it/s]                          


In [9]:
#Célula 9: Lançamento do WebApp

app = xpl.run_app(title_story="Monitor de Manutenção Preditiva - Indústria 5.0")

INFO:root:Your Shapash application run on http://AcerA5-JR:8050/
INFO:root:Use the method .kill() to down your app.


Exception in thread Thread-6 (<lambda>):
Traceback (most recent call last):
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\connection.py", line 204, in _new_conn
    sock = connection.create_connection(
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
OSError: [WinError 10049] O endereço solicitado não é válido no contexto

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\connectionpool.py", line 493, in _make_request
    conn.request(
  File "c:\XAILabProcess\.venv\lib\site-packages\urllib3\connection.py", line 500, in request
    self